In [1]:
import pandas_ta as ta
import pandas as pd
import numpy as np
import yfinance as yf
import vectorbt as vbt
from datetime import datetime
from tvDatafeed import TvDatafeed, Interval
import config_api
import matplotlib.pyplot as plt

In [2]:
user = config_api.tradingview_user
password = config_api.tradingview_password

In [3]:
tv = TvDatafeed(user, password, chromedriver_path= None)

(symbol: str, exchange: str = 'NSE', interval: Interval = Interval.in_daily, n_bars: int = 10, fut_contract: int | None = None, extended_session: bool = False) -> DataFrame)


In [4]:
symbol = ['ADAUSDT', 'BTCUSD']

In [5]:
ada_1h = tv.get_hist(symbol= symbol, exchange= 'BINANCE', interval= Interval.in_1_hour, n_bars= 5000)

In [4]:
ada_1d = tv.get_hist(symbol= 'ADAUSDT', exchange= 'BINANCE', interval= Interval.in_daily, n_bars= 5000)

In [4]:
btc_1d = tv.get_hist(symbol= 'BTCUSD', exchange= 'BINANCE', interval= Interval.in_daily, n_bars= 5000)

In [5]:
axs_1d = tv.get_hist(symbol= 'AXSUSD', exchange= 'BINANCE', interval= Interval.in_daily, n_bars= 5000)

In [ ]:
# pandas ta, เอา inds มาใส่ 
### obv, rsi, spt,
eth_1d

In [7]:
eth_1d = tv.get_hist(symbol= 'ETHUSD', exchange= 'BINANCE', interval= Interval.in_daily, n_bars= 5000)

In [6]:
df = axs_1d.copy()

In [65]:
strat = 'btc_obv20f40s_rsi>50'

In [9]:
df['rsi'] = df.ta.rsi(length=14)

In [7]:
df.ta.ma_obv(fast= 5, slow= 35, ma_type= 'ema', append= True)

,OBV,OBVe_20,OBVe_40,IBVe_20_40_diff
datetime,,,,
2020-01-03 07:00:00,3.558980,NaN,NaN,NaN
2020-01-04 07:00:00,1.729335,NaN,NaN,NaN
2020-01-05 07:00:00,5.094680,NaN,NaN,NaN
2020-01-06 07:00:00,15.344863,NaN,NaN,NaN
2020-01-07 07:00:00,44.862264,NaN,NaN,NaN
...,...,...,...,...
2022-01-25 07:00:00,-51833.761769,-48962.817023,-43114.818990,-5847.998034
2022-01-26 07:00:00,-54474.736809,-49487.761765,-43668.961322,-5818.800442
2022-01-27 07:00:00,-52349.339179,-49760.292947,-44092.394388,-5667.898559


In [8]:
df.ta.supertrend(multiplier= 3 ,append= True)

,SUPERT_7_3.0,SUPERTd_7_3.0,SUPERTl_7_3.0,SUPERTs_7_3.0
datetime,,,,
2020-01-03 07:00:00,0.000000,1,NaN,NaN
2020-01-04 07:00:00,NaN,1,NaN,NaN
2020-01-05 07:00:00,NaN,1,NaN,NaN
2020-01-06 07:00:00,NaN,1,NaN,NaN
2020-01-07 07:00:00,NaN,1,NaN,NaN
...,...,...,...,...
2022-01-25 07:00:00,42767.827496,-1,NaN,42767.827496
2022-01-26 07:00:00,42767.827496,-1,NaN,42767.827496
2022-01-27 07:00:00,42767.827496,-1,NaN,42767.827496


In [9]:
df.columns

Index(['symbol', 'open', 'high', 'low', 'close', 'volume', 'rsi', 'OBV',
       'OBVe_20', 'OBVe_40', 'IBVe_20_40_diff', 'SUPERT_7_3.0',
       'SUPERTd_7_3.0', 'SUPERTl_7_3.0', 'SUPERTs_7_3.0'],
      dtype='object')

ทำ signal มาลอง bactest ก่อน แล้วมาหา tune parameter ของ indys, signal อีกที

In [10]:
df.loc[df['rsi'] > 50, 'strength'] = 1
df.loc[df['rsi'] < 50, 'strength'] = -1


In [11]:
df.loc[df['IBVe_20_40_diff'] > 0, 'vol_in'] = 1
df.loc[df['IBVe_20_40_diff'] < 0, 'vol_in'] = -1

In [12]:
# supertrend ไม่ต้องทำ เพราะ direction มันมีvalue เป็น ซื้อ1 ขาย -1 ด้วยตัวมันเองอยู่แล้ว

In [13]:
# สร้าง signal ที่จะ feed ให้กับ vector_bt ทำเป็นตัว entry/exit
# df['signal'] = (df['vol_in'] == 1) & (df['strength'] == 1) & (df['SUPERTd_7_3.0'] == 1)
# df['signal'] = (df['strength'] == 1) & (df['SUPERTd_7_5.0'] == 1)
df['signal'] = (df['vol_in'] == 1) & (df['strength'] == 1)
## อนาคตหน้าพวก secondary signal (ตัวindy ต่างๆที่รวมเป็น signal ใหญ่) มีหลาย label ----> อาจจะใช้เป็นวิธี vol_in > 2 ไรงี้

In [14]:
df['signal']

datetime
2020-01-03 07:00:00    False
2020-01-04 07:00:00    False
2020-01-05 07:00:00    False
2020-01-06 07:00:00    False
2020-01-07 07:00:00    False
                       ...  
2022-01-25 07:00:00    False
2022-01-26 07:00:00    False
2022-01-27 07:00:00    False
2022-01-28 07:00:00    False
2022-01-29 07:00:00    False
Name: signal, Length: 758, dtype: bool

In [15]:
df.loc[df['signal'] == 1]

,symbol,open,high,low,close,volume,rsi,OBV,OBVe_20,OBVe_40,IBVe_20_40_diff,SUPERT_7_3.0,SUPERTd_7_3.0,SUPERTl_7_3.0,SUPERTs_7_3.0,strength,vol_in,signal
datetime,,,,,,,,,,,,,,,,,,
2020-02-11 07:00:00,BINANCE:BTCUSD,9830.635313,10351.248000,9670.973400,10221.811920,27.669135,72.190506,86.146267,46.390906,31.099175,15.291731,8936.202819,1,8936.202819,NaN,1.0,1.0,True
2020-02-12 07:00:00,BINANCE:BTCUSD,10286.173009,10485.680716,10090.767089,10302.680780,26.658223,73.087268,112.804490,52.716009,35.084800,17.631209,9013.506781,1,9013.506781,NaN,1.0,1.0,True
2020-02-13 07:00:00,BINANCE:BTCUSD,10324.399240,10543.274250,10001.115025,10221.163958,41.074392,70.615352,71.730098,54.526875,36.872375,17.654499,9013.506781,1,9013.506781,NaN,1.0,1.0,True
2020-02-14 07:00:00,BINANCE:BTCUSD,10200.616420,10411.720099,10003.034246,10343.408100,27.715841,72.137236,99.445939,58.804881,39.924744,18.880136,9013.506781,1,9013.506781,NaN,1.0,1.0,True
2020-02-15 07:00:00,BINANCE:BTCUSD,10353.985920,10411.720099,9771.728500,9898.056840,29.339565,59.954559,70.106374,59.881213,41.397019,18.484194,9013.506781,1,9013.506781,NaN,1.0,1.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-11-11 07:00:00,BINANCE:BTCUSD,65081.692207,65948.622623,64053.062168,65134.627448,1637.024560,60.102656,-17514.995484,-16916.113584,-19062.944584,2146.831000,58614.025012,1,58614.025012,NaN,1.0,1.0,True
2021-11-12 07:00:00,BINANCE:BTCUSD,65302.416130,65962.005794,62507.685018,64427.343482,2423.896680,57.735620,-19938.892164,-17203.997258,-19105.673734,1901.676476,58614.025012,1,58614.025012,NaN,1.0,1.0,True
2021-11-13 07:00:00,BINANCE:BTCUSD,64588.188458,65589.356293,63637.818584,64773.486501,1552.793190,58.595049,-18386.098974,-17316.578374,-19070.572526,1753.994152,58614.025012,1,58614.025012,NaN,1.0,1.0,True


In [16]:
df_signal = df[['open', 'close', 'rsi', 'IBVe_20_40_diff', 'vol_in', 'strength', 'signal']] # 'vol_in'

In [17]:
#df_signal.loc[df_signal['SUPERTd_7_5.0'] == 1]
df_signal.loc[df_signal['signal'] == 0]

,open,close,rsi,IBVe_20_40_diff,vol_in,strength,signal
datetime,,,,,,,
2020-01-03 07:00:00,6976.103033,7330.636993,NaN,NaN,NaN,NaN,False
2020-01-04 07:00:00,7327.604522,7330.023198,NaN,NaN,NaN,NaN,False
2020-01-05 07:00:00,7363.287571,7353.258110,NaN,NaN,NaN,NaN,False
2020-01-06 07:00:00,7329.957516,7740.114359,NaN,NaN,NaN,NaN,False
2020-01-07 07:00:00,7771.116424,8158.722900,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...
2022-01-25 07:00:00,36722.270000,37006.030000,30.337674,-5847.998034,-1.0,-1.0,False
2022-01-26 07:00:00,37010.970000,36879.360000,30.031552,-5818.800442,-1.0,-1.0,False
2022-01-27 07:00:00,36874.190000,37197.990000,31.893213,-5667.898559,-1.0,-1.0,False


In [18]:
df_signal['next_open'] = df_signal['open'].shift(-1)

<ipython-input-18-aa3ab63ed215>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_signal['next_open'] = df_signal['open'].shift(-1)


In [19]:
df_signal

,open,close,rsi,IBVe_20_40_diff,vol_in,strength,signal,next_open
datetime,,,,,,,,
2020-01-03 07:00:00,6976.103033,7330.636993,NaN,NaN,NaN,NaN,False,7327.604522
2020-01-04 07:00:00,7327.604522,7330.023198,NaN,NaN,NaN,NaN,False,7363.287571
2020-01-05 07:00:00,7363.287571,7353.258110,NaN,NaN,NaN,NaN,False,7329.957516
2020-01-06 07:00:00,7329.957516,7740.114359,NaN,NaN,NaN,NaN,False,7771.116424
2020-01-07 07:00:00,7771.116424,8158.722900,NaN,NaN,NaN,NaN,False,8200.589385
...,...,...,...,...,...,...,...,...
2022-01-25 07:00:00,36722.270000,37006.030000,30.337674,-5847.998034,-1.0,-1.0,False,37010.970000
2022-01-26 07:00:00,37010.970000,36879.360000,30.031552,-5818.800442,-1.0,-1.0,False,36874.190000
2022-01-27 07:00:00,36874.190000,37197.990000,31.893213,-5667.898559,-1.0,-1.0,False,37179.320000


In [20]:
signal_vectorbt = df.ta.tsignals(df_signal['signal'], asbool= True)

In [21]:
df_signal.ta.tsignals(df_signal['signal'], asbool= True, append= True)

C:\Anaconda3\lib\site-packages\pandas_ta\core.py:420: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = result.iloc[:, i]


,TS_Trends,TS_Trades,TS_Entries,TS_Exits
datetime,,,,
2020-01-03 07:00:00,False,0,False,False
2020-01-04 07:00:00,False,0,False,False
2020-01-05 07:00:00,False,0,False,False
2020-01-06 07:00:00,False,0,False,False
2020-01-07 07:00:00,False,0,False,False
...,...,...,...,...
2022-01-25 07:00:00,False,0,False,False
2022-01-26 07:00:00,False,0,False,False
2022-01-27 07:00:00,False,0,False,False


In [22]:
signal_vectorbt

,TS_Trends,TS_Trades,TS_Entries,TS_Exits
datetime,,,,
2020-01-03 07:00:00,False,0,False,False
2020-01-04 07:00:00,False,0,False,False
2020-01-05 07:00:00,False,0,False,False
2020-01-06 07:00:00,False,0,False,False
2020-01-07 07:00:00,False,0,False,False
...,...,...,...,...
2022-01-25 07:00:00,False,0,False,False
2022-01-26 07:00:00,False,0,False,False
2022-01-27 07:00:00,False,0,False,False


In [23]:
signal_vectorbt.loc[signal_vectorbt['TS_Trades'] != 0]

,TS_Trends,TS_Trades,TS_Entries,TS_Exits
datetime,,,,
2020-02-11 07:00:00,True,1,True,False
2020-02-25 07:00:00,False,-1,False,True
2020-03-06 07:00:00,True,1,True,False
2020-03-07 07:00:00,False,-1,False,True
2020-04-02 07:00:00,True,1,True,False
2020-04-13 07:00:00,False,-1,False,True
2020-04-16 07:00:00,True,1,True,False
2020-04-20 07:00:00,False,-1,False,True
2020-04-22 07:00:00,True,1,True,False


In [24]:
port = vbt.Portfolio.from_signals(
    df_signal['close'],
    entries= signal_vectorbt.TS_Entries,
    exits= signal_vectorbt.TS_Exits,
    freq= '1D',
    init_cash= 100,
    fees= 0.0025,
    slippage= 0.0050
)

In [25]:
port.plot().show()

In [26]:
port.get_drawdowns().plot().show()

In [27]:
return_adjust_vol = port.annualized_return() / port.annualized_volatility()
print(f'return adjust vol: {return_adjust_vol}')


return adjust vol: 2.0587999325779407


In [28]:
port.alpha()

0.45884141937910417

In [29]:
port.plot_underwater()

FigureWidget({
    'data': [{'fill': 'tozeroy',
              'fillcolor': 'rgba(220,57,18,0.3000)',
         …

In [30]:
port.annual_returns()

datetime
2020-01-03    1.146932
2021-01-02    0.950736
2022-01-02    0.000000
Freq: 365D, dtype: float64

In [42]:
port.stats()

Start                               2020-01-03 07:00:00
End                                 2022-01-29 07:00:00
Period                                758 days 00:00:00
Start Value                                       100.0
End Value                                    418.809785
Total Return [%]                             318.809785
Benchmark Return [%]                         415.290418
Max Gross Exposure [%]                            100.0
Total Fees Paid                               17.732136
Max Drawdown [%]                              25.905751
Max Drawdown Duration                 182 days 00:00:00
Total Trades                                         22
Total Closed Trades                                  22
Total Open Trades                                     0
Open Trade P&L                                      0.0
Win Rate [%]                                  36.363636
Best Trade [%]                               167.276877
Worst Trade [%]                              -10

In [177]:
# use variable that was set as the string of strategy feature
port_df = pd.DataFrame(port.stats()).rename(columns={0: strat})


In [180]:
### only first time ( creating csv )
#port_df.to_csv('crypto_tf1d_backtest.csv', index= True)

In [184]:
#### if csv file is exist use this script
port_csv = pd.read_csv('./crypto_tf1d_backtest.csv', index_col=0)
port_csv[f'{strat}'] = port_df
port_csv.to_csv('crypto_tf1d_backtest.csv', index= True)

In [185]:
port_csv

,strat,btc_obv20f40s_rsi>50
Start,2020-01-03 07:00:00,2020-01-03 07:00:00
End,2022-01-29 07:00:00,2022-01-29 07:00:00
Period,758 days 00:00:00,758 days 00:00:00
Start Value,100.0,100.0
End Value,418.8097853581229,418.809785
Total Return [%],318.8097853581229,318.809785
Benchmark Return [%],415.2904179761759,415.290418
Max Gross Exposure [%],100.0,100.0
Total Fees Paid,17.732136263696717,17.732136
Max Drawdown [%],25.90575124860386,25.905751


In [173]:
port_csv[f'1{strat}']

Start                               2020-01-03 07:00:00
End                                 2022-01-29 07:00:00
Period                                758 days 00:00:00
Start Value                                       100.0
End Value                                    418.809785
Total Return [%]                             318.809785
Benchmark Return [%]                         415.290418
Max Gross Exposure [%]                            100.0
Total Fees Paid                               17.732136
Max Drawdown [%]                              25.905751
Max Drawdown Duration                 182 days 00:00:00
Total Trades                                         22
Total Closed Trades                                  22
Total Open Trades                                     0
Open Trade P&L                                      0.0
Win Rate [%]                                  36.363636
Best Trade [%]                               167.276877
Worst Trade [%]                              -10

In [163]:
port_csv

,strat_new3
strat_new2,
2020-01-03 07:00:00,NaN
2022-01-29 07:00:00,NaN
758 days 00:00:00,NaN
100.0,NaN
418.8097853581229,NaN
318.8097853581229,NaN
415.2904179761759,NaN
100.0,NaN
17.732136263696717,NaN


เมื่อได้ parameter ที่ดีแล้วค่อยนำมาทำ label แล้วจูน randomforest feature อีกที

In [36]:
action = df_signal.loc[df_signal['TS_Trades'] != 0]

In [37]:
action = action[['open', 'rsi', 'IBVe_20_40_diff', 'SUPERTd_7_3.0', 'vol_in', 'strength', 'TS_Trades', 'close']]

KeyError: "['SUPERTd_7_3.0'] not in index"

In [ ]:
action.loc['2020-10':]

,open,rsi,IBVe_20_40_diff,SUPERTd_7_3.0,vol_in,strength,TS_Trades,close
datetime,,,,,,,,
2020-10-02 07:00:00,0.09803,45.755842,1.723419e+08,1,1.0,-1.0,-1,0.09286
2020-10-04 07:00:00,0.09332,50.036630,1.690596e+08,1,1.0,1.0,1,0.09686
2020-10-06 07:00:00,0.09740,45.888881,1.732385e+08,1,1.0,-1.0,-1,0.09289
2020-10-09 07:00:00,0.09632,55.250775,2.117100e+08,1,1.0,1.0,1,0.10129
2020-10-26 07:00:00,0.10640,48.918548,3.338067e+08,1,1.0,-1.0,-1,0.10306
2020-11-13 07:00:00,0.10462,55.313049,5.237510e+06,1,1.0,1.0,1,0.10629
2020-11-15 07:00:00,0.10411,47.559100,-1.036859e+06,1,-1.0,-1.0,-1,0.10046
2020-11-16 07:00:00,0.10045,51.921027,2.162489e+06,1,1.0,1.0,1,0.10387
2020-12-08 07:00:00,0.15401,49.822325,1.532837e+09,1,1.0,-1.0,-1,0.14120


In [ ]:
[2021-03-15:2021-07-07]

In [ ]:
df_action.columns

Index(['symbol', 'open', 'high', 'low', 'close', 'volume', 'OBV', 'OBVe_2',
       'OBVe_10', 'IBVe_2_10_diff', 'rsi', 'SUPERT_7_3.0', 'SUPERTd_7_3.0',
       'SUPERTl_7_3.0', 'SUPERTs_7_3.0', 'strength', 'vol_in', 'signal',
       'TS_Trends', 'TS_Trades', 'TS_Entries', 'TS_Exits'],
      dtype='object')

In [ ]:
# ทำ df แยกมาเพื่อเรียกดู signal ต่างๆ ว่าปัญหามาจากตรงไหน

df_action = df_action[[ 'volume', 'OBV', 'OBVe_2',
       'OBVe_10', 'IBVe_2_10_diff', 'rsi',
       'SUPERTl_7_3.0', 'SUPERTs_7_3.0', 'SUPERTd_7_3.0', 'strength', 'vol_in', 'signal', 'close',]]

เหมือนปัญหาจะมาจากที่ vol_in มันอ่อนเกินไป (2tf ตัดกับ 5tf)

In [ ]:
# เรียกดู orders ที่ออกเพราะ vol
df_action.loc[df_action['vol_in'] == -1].head(50)

เกือบทั้งหมด 80% ปิดเพราะ vol_in